In [1]:
import csv
import numpy as np
from typing import Tuple
from numpy import ndarray
import random
from tqdm import tqdm

In [2]:
csv_file=open('WineQT.csv', 'r')
csv_reader = csv.reader(csv_file)
data=[]
for row in csv_reader:
    row.pop(len(row)-1)
    data.append(row)
att=data[0]
val_size=(len(data)-1)//10
test_size=val_size

In [3]:
val_set=[]
test_set=[]
while len(val_set)<val_size:
    inst=random.randint(1,len(data)-1)
    if inst not in val_set:
        val_set.append(inst)
        
while len(test_set)<val_size:
    inst=random.randint(1,len(data)-1)
    if inst not in val_set and inst not in test_set:
        test_set.append(inst)


In [4]:
train_data=[]
val_data=[]
test_data=[]

for i in range(1,len(data)):
    if i in val_set:
        val_data.append(data[i])
    elif i in test_set:
        test_data.append(data[i])
    else:
        train_data.append(data[i])

print(len(train_data))
print(len(val_data))
print(len(test_data))

915
114
114


In [5]:
def create_dataset(data):
    dataset={}
    dataset['input']=[]
    dataset['label']=[]
    for x in data:
        y=np.zeros(11)
        y[int(x[-1])]=1
        dataset['label'].append(y)
        inp=[]
        
        for val in x[:-1]:
           
            inp.append(float(val))
        dataset['input'].append(np.array(inp))
    return dataset

In [6]:
train_dataset=create_dataset(train_data)
print(train_dataset['input'][0])
print(train_dataset['label'][0])
val_dataset=create_dataset(val_data)
test_dataset=create_dataset(test_data)

[ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
  0.68    9.8   ]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [8]:
epochs=10000
lr=0.1
w=np.random.rand(11,11)
b=np.random.rand(11)
val_res=[]
max_val=0
noupdate=0

In [9]:
varia=tqdm(range(epochs))
for epoch in varia:
    for index in range(len(train_dataset['input'])):
        x=train_dataset['input'][index]
        y=train_dataset['label'][index]
        #print(x)
        z=w.T@x+b
        #print(w)
        y_pred=softmax(z)
        grad=y_pred-y
        grad_w=grad@x.T
        grad_b=grad
        w=w-lr*grad_w
        b=b-lr*grad_b
    good=0
    for index in range(len(val_dataset['input'])):
        x=val_dataset['input'][index]
        y=val_dataset['label'][index]
        z=w.T@x+b
        y_pred=softmax(z)
        y_pred=np.argmax(y_pred)
        y=np.argmax(y)
        if y_pred==y:
            good+=1
    #print(good)
    val_res.append(good/len(val_dataset['input']))
    if max_val< good/len(val_dataset['input']):
        max_val=good/len(val_dataset['input'])
        noupdate=0
    else:
        noupdate+=1
    if noupdate==100:
        lr=lr*0.8
        noupdate=0
    
    
        
        
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:25<00:00, 68.94it/s]


In [10]:
print(max_val)

0.4824561403508772


In [11]:
good=0
for index in range(len(test_dataset['input'])):
    x=test_dataset['input'][index]
    y=test_dataset['label'][index]
    z=w.T@x+b
    y_pred=softmax(z)
    y_pred=np.argmax(y_pred)
    y=np.argmax(y)
    if y_pred==y:
        good+=1
print(good/len(val_dataset['input']))

0.45614035087719296
